# Inicializar

In [11]:
import gurobipy as gp
from gurobipy import GRB
import os
import re
import pandas as pd

In [12]:
def mostrarResultadosTabla(cantidadNodos, costo, variablesDecision, tipo="general"):
	"""
	Función genérica para mostrar resultados de optimización en tablas.

	Parámetros:
	- cantidadNodos (int): Número de nodos en el modelo.
	- costo (float): Costo total de la solución.
	- variablesDecision (dict): Diccionario con las variables de decisión y sus valores.
	- tipo (str): Tipo de modelo ("general" o "hibrido").
	"""
	print("=" * 52)
	print(f"Cantidad de Nodos: {cantidadNodos}")
	print("=" * 52)

	if costo is None:
		print("No se encontró solución")
		return

	print("Resultado de la Optimización:")
	print("=" * 52)
	print(f"Costo Total: {costo}")
	print(f"Costo nodos: {variablesDecision.get('nodesCost', 'N/A')}")
	print(f"Costo enlaces: {variablesDecision.get('linksCost', 'N/A')}")
	print("=" * 52)

	# Filtrar variables de decisión según prefijo
	xVars = {var: val for var, val in variablesDecision.items() if var.startswith("x")}
	yVars = {var: val for var, val in variablesDecision.items() if var.startswith("y")}

	# Procesar nodos activos
	xactiveNodes = procesarVariablesActivas(xVars, cantidadNodos, "x")
	if tipo == "hibrido":
		yactiveNodes = procesarVariablesActivas(yVars, cantidadNodos, "y")

	# Crear tablas
	columns_titles_x = ["Low Cost", "Mid Cost", "High Cost"]
	row_index = [u + 1 for u in range(cantidadNodos)]
	tablax = pd.DataFrame(xactiveNodes, columns=columns_titles_x, index=row_index)
	print("Nodos activos (x):")
	print(tablax)
	print("=" * 52)

	if tipo == "hibrido":
		columns_titles_y = [f"Subred {i}" for i in range(len(yactiveNodes[0]))]
		tablay = pd.DataFrame(yactiveNodes, columns=columns_titles_y, index=row_index)
		print("Nodos activos (y):")
		print(tablay)
		print("=" * 52)

def procesarVariablesActivas(variables, cantidadNodos, prefix):
    """
    Procesa las variables activas para construir una lista de valores por nodo.

    Parámetros:
    - variables (dict): Diccionario con las variables de decisión y sus valores.
    - cantidadNodos (int): Número de nodos en el modelo.
    - prefix (str): Prefijo de las variables ("x" o "y").

    Retorna:
    - List[List[int]]: Lista de listas con los valores de las variables activas.
    """
    activeNodes = []
    for u in range(cantidadNodos):
        valores = []
        pattern = re.compile(rf'^{prefix}\[{u},')
        for var, val in variables.items():
            if pattern.match(var):
                valores.append(int(val))
        activeNodes.append(valores)
    return activeNodes

# Modelo base

In [13]:
def base_model(N, c_values):
	"""
	Crea un modelo base de optimización para desplegar nodos con diferentes costos y confiabilidades.

	Parámetros:
	- N (int): Número de nodos a desplegar. Debe ser al menos 4.

	Retorna:
	- model (gurobipy.Model): Modelo optimizado de Gurobi.
	"""
	if N < 4:
		raise ValueError("El número de nodos debe ser al menos 4.")

	# Crear el modelo
	model = gp.Model(f"General_Model_{N}_Nodes")

	# Conjunto de nodos
	U = range(N)  # Nodos a desplegar
	I = range(len(c_values))  # Tipos de nodos (Low, Medium, High)

	# Variables de decisión x[u, i]
	# i = 0: Low, i = 1: Medium, i = 2: High
	# x[u, i] = 1 si el nodo u de tipo i está activo, 0 en caso contrario
	x = model.addVars(U, I, vtype=GRB.BINARY, name="x")

	# Costo total de los nodos desplegados
	# c_values[i] = costo del nodo de tipo i
	# nodesCost = suma de los costos de los nodos desplegados
	nodesCost = model.addVar(vtype=GRB.CONTINUOUS, name="nodesCost")
	model.addConstr(nodesCost == gp.quicksum(c_values[i] * x[u, i] for i in I for u in U), name="NodesCost_def")

	# Costo total de los enlaces (inicialmente 0 en el modelo base)
	# linksCost = suma de los costos de los enlaces desplegados
	# linksCost = 0 en el modelo base
	# linksCost varía dependiendo del modelo específico
	linksCost = model.addVar(vtype=GRB.CONTINUOUS, name="linksCost")
	model.addConstr(linksCost == 0, name="LinksCost_General")

	# Restricción: Cada nodo debe ser de un único tipo i
	model.addConstrs(
		(gp.quicksum(x[u, i] for i in I) == 1 for u in U),
		name="Unicidad_i"
	)

	# Configurar el objetivo: Minimizar el costo total
	# nodesCost + linksCost
	# linksCost = 0 en el modelo base, por lo que solo se minimiza nodesCost
	model.setObjective(nodesCost + linksCost, GRB.MINIMIZE)

	# Optimizar el modelo
	model.setParam('OutputFlag', 0)  # Desactivar salida de Gurobi
	model.optimize()

	return model

# Modelos heredados

In [14]:
def serie_model(model_base, N, L):
    """
    Extiende el modelo base para incluir las restricciones y costos específicos del modelo en serie.

    Parámetros:
    - model_base (gurobipy.Model): Modelo base generado por base_model.
    - N (int): Número de nodos en la red.
    - L (float): Costo de un enlace.

    Retorna:
    - costo_total (float): Costo total de la solución.
    - variables_decision (dict): Diccionario con las variables de decisión y sus valores.
    """
    # Copiar el modelo base
    model = model_base.copy()

    # Recuperar la variable linksCost del modelo base
    linksCost = model.getVarByName("linksCost")
    if linksCost is None:
        raise ValueError("No se encontró la variable de costo de enlaces en el modelo base.")

    # Eliminar la restricción general de linksCost (si existe)
    # Esto es necesario para evitar conflictos al agregar la nueva restricción
    # linksCost = 0 en el modelo base, pero en el modelo en serie debe ser L * (N - 1)
    linksCost_Condition = model.getConstrByName("LinksCost_General")
    if linksCost_Condition is not None:
        model.remove(linksCost_Condition)

    # Agregar la restricción específica del modelo en serie para el costo de enlaces
    # linksCost = L * (N - 1)
    # donde L es el costo de un enlace y N es el número de nodos
    model.addConstr(linksCost == L * (N - 1), name="LinksCost_Serie")

    # Optimizar el modelo
    model.optimize()

    # Verificar si se encontró una solución óptima
    if model.status == GRB.OPTIMAL:
        # Extraer las variables de decisión y sus valores
        variables_decision = {var.varName: var.x for var in model.getVars()}
        return model.objVal, variables_decision
    else:
        raise Exception("No se encontró una solución óptima.")

In [15]:
def parallel_model(model_base, N, L):
	"""
	Extiende el modelo base para incluir las restricciones y costos específicos del modelo paralelo.

	Parámetros:
	- model_base (gurobipy.Model): Modelo base generado por base_model.
	- N (int): Número de nodos en la red.
	- L (float): Costo de un enlace.

	Retorna:
	- costo_total (float): Costo total de la solución.
	- variables_decision (dict): Diccionario con las variables de decisión y sus valores.
	"""
	# Copiar el modelo base
	model = model_base.copy()

	# Recuperar la variable linksCost del modelo base
	linksCost = model.getVarByName("linksCost")
	if linksCost is None:
		raise ValueError("No se encontró la variable de costo de enlaces en el modelo base.")

	# Eliminar la restricción general de linksCost (si existe)
	linksCost_Condition = model.getConstrByName("LinksCost_General")
	if linksCost_Condition is not None:
		model.remove(linksCost_Condition)

	# Agregar la restricción específica del modelo paralelo para el costo de enlaces
	# En una red en paralelo, todos los nodos están conectados entre sí
	# Número de enlaces = N * (N - 1) / 2
	model.addConstr(2*linksCost == L * N * (N - 1), name="LinksCost_Paralelo")

	# Optimizar el modelo
	model.optimize()

	# Verificar si se encontró una solución óptima
	if model.status == GRB.OPTIMAL:
		# Extraer las variables de decisión y sus valores
		variables_decision = {var.varName: var.x for var in model.getVars()}
		return model.objVal, variables_decision
	else:
		raise Exception("No se encontró una solución óptima.")

In [16]:
def hibrid_model(model_base, N, L):
    """
    Extiende el modelo base para incluir las restricciones y costos específicos del modelo híbrido.

    Parámetros:
    - model_base (gurobipy.Model): Modelo base generado por base_model.
    - N (int): Número de nodos en la red.
    - L (float): Costo de un enlace.

    Retorna:
    - costo_total (float): Costo total de la solución.
    - variables_decision (dict): Diccionario con las variables de decisión y sus valores.
    - model (gurobipy.Model): Modelo optimizado de Gurobi.
    """
    if N < 4:
        raise ValueError("El número de nodos debe ser al menos 4.")

    # Conjuntos
    U = range(N)  # Nodos a desplegar
    J = range(N // 3 + 1)  # Subredes (0: serie, 1 en adelante: paralelo)

    # Copiar el modelo base
    model = model_base.copy()

    # Recuperar la variable linksCost del modelo base
    linksCost = model.getVarByName("linksCost")
    if linksCost is None:
        raise ValueError("No se encontró la variable de costo de enlaces en el modelo base.")

    # Eliminar la restricción general de linksCost (si existe)
    linksCost_Condition = model.getConstrByName("LinksCost_General")
    if linksCost_Condition is not None:
        model.remove(linksCost_Condition)

    # Variables adicionales
    y = model.addVars(U, J, vtype=GRB.BINARY, name="y")  # Nodo u pertenece a la subred j
    alpha = model.addVars(J, vtype=GRB.BINARY, name="alpha")  # Subred j activa o no
    p = model.addVars(J, vtype=GRB.INTEGER, name="p")  # Número de nodos en la subred j
    z = model.addVars(J, vtype=GRB.INTEGER, name="z")  # Número de enlaces en paralelo por subred j
    N_s = model.addVar(vtype=GRB.INTEGER, name="N_s")  # Número de nodos en la subred serie (j = 0)

    # ============================
    # Definiciones de variables auxiliares
    # ============================

    # Definición: Número de nodos en la subred serie (j = 0)
    model.addConstr(N_s == gp.quicksum(y[u, 0] for u in U), name="Ns_def")

    # Definición: Número de nodos en cada subred paralela (j >= 1)
    model.addConstrs(
        (p[j] == gp.quicksum(y[u, j] for u in U) for j in J if j > 0),
        name="p_def"
    )

    # Definición: Número de enlaces en paralelo por subred j >= 1
    model.addConstrs(
        (2 * z[j] == p[j] * (p[j] - 1) for j in J if j > 0),
        name="Enlaces_Paralelo_Subred"
    )

    # ============================
    # Restricciones del modelo
    # ============================

    # Restricción: Cada nodo debe pertenecer a una única subred
    model.addConstrs(
        (gp.quicksum(y[u, j] for j in J) == 1 for u in U),
        name="Unicidad_Subred"
    )

    # Restricción: Activar subred si tiene nodos asignados
    model.addConstrs(
        (alpha[j] >= y[u, j] for u in U for j in J if j > 0),
        name="Activar_Subred"
    )

    # Restricción: Si la subred j >= 1 existe, debe tener al menos 3 nodos
    model.addConstrs(
        (gp.quicksum(y[u, j] for u in U) >= 3 * alpha[j] for j in J if j > 0),
        name="Subredes_Min_3"
    )

    # Restricción: Al menos una subred paralela debe estar activa
    model.addConstr(
        gp.quicksum(alpha[j] for j in J if j > 0) >= 1,
        name="AlMenosUnaSubredActiva"
    )

    # ============================
    # Cálculo del costo de enlaces
    # ============================

    # H: Número de enlaces adicionales entre subredes paralelas
    H = gp.quicksum(alpha[j] for j in J if j > 0) - 1  # Enlaces adicionales por subredes paralelas

    # B: Número de enlaces internos en las subredes paralelas
    B = gp.quicksum(z[j] for j in J if j > 0)

    # Restricción: Costo total de los enlaces
    model.addConstr(
        linksCost == L * (N_s + H + B),
        name="LinksCost_Hibrido"
    )

    # Optimizar el modelo
    model.optimize()

    # Verificar si se encontró una solución óptima
    if model.status == GRB.OPTIMAL:
        # Extraer las variables de decisión y sus valores
        variables_decision = {var.varName: var.x for var in model.getVars()}
        return model.objVal, variables_decision, model
    else:
        raise Exception("No se encontró una solución óptima.")

# Pruebas

In [17]:
N = 10
L = 10
c_values = {0: 5, 1: 12, 2: 20}
confiabilidad_values = {0: 0.9, 1: 0.95, 2: 0.99}
modeloBase = base_model(N, c_values)

In [18]:
try:
	costoSerie, variablesDecisionSerie = serie_model(modeloBase, N, L)
	print("Modelo serie")
	mostrarResultadosTabla(N, costoSerie, variablesDecisionSerie)
except Exception as e:
	print(f"Error: {e}")

Modelo serie
Cantidad de Nodos: 10
Resultado de la Optimización:
Costo Total: 140.0
Costo nodos: 50.0
Costo enlaces: 90.0
Nodos activos (x):
    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0


In [19]:
try:
	costoParalelo, variablesDecisionParalelo = parallel_model(modeloBase, N, L)
	print("\nModelo solo paralelo")
	mostrarResultadosTabla(N, costoParalelo, variablesDecisionParalelo)
except Exception as e:
	print(f"Error: {e}")


Modelo solo paralelo
Cantidad de Nodos: 10
Resultado de la Optimización:
Costo Total: 500.0
Costo nodos: 50.0
Costo enlaces: 450.0
Nodos activos (x):
    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0


In [20]:
try:
	costoHibrido, variablesDecisionHibrido, modeloHibrido = hibrid_model(modeloBase, N, L)
	print("Modelo híbrido")
	mostrarResultadosTabla(N, costoHibrido, variablesDecisionHibrido, "hibrido")
except Exception as e:
	print(f"Error: {e}")

Modelo híbrido
Cantidad de Nodos: 10
Resultado de la Optimización:
Costo Total: 150.0
Costo nodos: 50.0
Costo enlaces: 100.0
Nodos activos (x):
    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0
Nodos activos (y):
    Subred 0  Subred 1  Subred 2  Subred 3
1          0         0         1         0
2          0         0         1         0
3          1         0         0         0
4          1         0         0         0
5          0         0         1         0
6          1         0         0         0
7          1         0         0         0
8          1         0         0         0
9          1         0         0         0
10         1         0         0 